In [438]:
import pandas as pd
import re
import string

In [439]:
data =pd.read_excel('Ozurgeti.xlsx')                   

In [440]:
data.head(4)
data.columns

Index(['საიდენტიფიკაციო ნომერი', 'პირადი ნომერი',
       'ორგანიზაციულ-სამართლებრივი ფორმა', 'სუბიექტის დასახელება',
       'რეგიონი (იურიდიული)', 'მუნიციპალიტეტი/ქალაქი (იურიდიული)',
       'დაბის/თემის/სოფლის საკრებულო (იურიდიული)', 'დაბა/სოფელი (იურიდიული)',
       'იურიდიული მისამართი', 'რეგიონი (ფაქტობრივი)',
       'მუნიციპალიტეტი/ქალაქი (ფაქტობრივი)',
       'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)', 'დაბა/სოფელი (ფაქტობრივი)',
       'ფაქტობრივი მისამართი', 'საქმიანობის კოდი NACE Rev.2',
       'საქმიანობის დასახელება NACE Rev.2', 'პარტნიორი', 'ხელმძღვანელი',
       'ტელეფონი', 'ელფოსტა', 'საკუთრების ფორმა',
       'აქტიური ეკონომიკური სუბიექტები', 'ბიზნესის ზომა'],
      dtype='object')

In [441]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,რეგიონი (იურიდიული),მუნიციპალიტეტი/ქალაქი (იურიდიული),დაბის/თემის/სოფლის საკრებულო (იურიდიული),დაბა/სოფელი (იურიდიული),იურიდიული მისამართი,რეგიონი (ფაქტობრივი),...,ფაქტობრივი მისამართი,საქმიანობის კოდი NACE Rev.2,საქმიანობის დასახელება NACE Rev.2,პარტნიორი,ხელმძღვანელი,ტელეფონი,ელფოსტა,საკუთრების ფორმა,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,იმ,ზურაბ შელეგია,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,ქ. თბილისის ნაძალადევის რაიონში / მანჯგალაძის 29,გურია,...,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",ზურაბ შელეგია - 100,ზურაბ შელეგია,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
1,100019383,6.200100e+10,იმ,მარინე ჭელიძე,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,"ქ. თბილისის ნაძალადევის რაიონში / სანზონა კ.6,ბ.1",გურია,...,NaN,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",მარინე ჭელიძე - 100,მარინე ჭელიძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
2,100085300,3.300100e+10,იმ,დავით ანთაძე,გურია,ოზურგეთის მუნიციპალიტეტი,ბახვის თემის საკრებულო,NaN,ბახვის თემის საკრებულო,გურია,...,ბახვის თემის საკრებულო,NaN,NaN,დავით ანთაძე - 100,დავით ანთაძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN


In [442]:
data = data[['საიდენტიფიკაციო ნომერი',
             'პირადი ნომერი',
             'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)',
             'ორგანიზაციულ-სამართლებრივი ფორმა',
             'სუბიექტის დასახელება','ფაქტობრივი მისამართი',
             'საქმიანობის დასახელება NACE Rev.2',
             'აქტიური ეკონომიკური სუბიექტები',
              'ბიზნესის ზომა']]

In [443]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,NaN,იმ,ზურაბ შელეგია,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
1,100019383,6.200100e+10,ურეკის სადაბო საკრებულო,იმ,მარინე ჭელიძე,NaN,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
2,100085300,3.300100e+10,ბახვის თემის საკრებულო,იმ,დავით ანთაძე,ბახვის თემის საკრებულო,NaN,NaN,NaN
3,100096389,3.300101e+10,NaN,იმ,სოსო სადრაძე,შემოქმედის,"სხვა ხილის, კენკროვნების და კაკლისებრების მოყვანა",NaN,NaN


In [444]:
data = data[(data['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'].str.contains('ქ. ოზურგეთი', na=False)) ]


In [445]:
data = data[~(data['ფაქტობრივი მისამართი'].isnull())]


In [446]:
def clean_text(arg):
    cleaned = re.sub(r'[^\w\s]',' ',arg)
    return cleaned


def remove_punctuation(input_string):
    result = input_string
    for char in string.punctuation:
        result = result.replace(char, ' ')
    return result


In [447]:
data['ნომერი'] = data['ფაქტობრივი მისამართი'].apply(lambda x: "".join([i for i in x if i.isnumeric()]))

In [448]:
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(clean_text)
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(remove_punctuation)

In [449]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub('\d+', '',i)  for i in x ]))

In [450]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub(r'\\N', '',i)  for i in x ]))

In [451]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ''.join([i for i in x if i != 'N']))

In [452]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i.isnumeric() or len(i) >2]))

In [453]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i.isnumeric() or len(i) >3]))

In [454]:
data = data[(data['ნომერი']!='') & (data['ქუჩა']!='')]

In [455]:
data.shape

(715, 11)

In [ ]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i not in [ 'ქუჩა',
                                                                                        'საქართველო',
                                                                                        'ოზურგეთი',
                                                                                        'ოზურგეთის',
                                                                                        'რაიონში',
                                                                                        'სახელობის'
                                                                                        ]
                                                                                    ]
                                                                                )
                                                                            )

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა
53,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,3,ბათუმის
83,105211994,1.007003e+09,ქ. ოზურგეთი,იმ,მადალენა თოიძე,გურამიშვილის ქ.12,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,12,გურამიშვილის
141,131951583,2.100100e+10,ქ. ოზურგეთი,იმ,ზაალი ფანჩულიძე,ე. თაყაიშვილის I შეს. N1,უალკოჰოლო არომატიზებული და/ან დამტკბარი სასმელ...,NaN,NaN,1,თაყაიშვილის
165,137059778,3.300101e+10,ქ. ოზურგეთი,იმ,ნაირა მეგრელიშვილი,გურიის ქ.N33,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,33,გურიის
185,137061104,3.300107e+10,ქ. ოზურგეთი,იმ,იური მჟავია,გურიის ქუჩა #35,საკუთარი ან იჯარით აღებული საცხოვრებელი შენობე...,NaN,NaN,35,გურიის
190,137061300,3.300102e+10,ქ. ოზურგეთი,იმ,აკაკი სირაძე,აღმაშენებლის ქ.N77,სხვა ავეჯის წარმოება,NaN,NaN,77,აღმაშენებლის
195,137061382,3.300102e+10,ქ. ოზურგეთი,იმ,თამარ შავიშვილი -ენძელა,`დავით`აღმაშენებლის`ქ.`Â¿129`,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,129,დავით აღმაშენებლის
209,137061747,3.300101e+10,ქ. ოზურგეთი,იმ,ოლეგი ბაკურაძე,გურიის 35,საცალო ვაჭრობა ტანსაცმლით სპეციალიზებულ მაღაზი...,NaN,NaN,35,გურიის
213,137061916,3.300102e+10,ქ. ოზურგეთი,იმ,ჟულერ ვასილევი,თაყაიშვილის 37,პურის წარმოება,აქტიური,მცირე,37,თაყაიშვილის
217,137062011,NaN,ქ. ოზურგეთი,იმ,ვაჟა ცხომელიძე,გურიის ქ. №35,"საცალო ვაჭრობა რკინა-კავეულით, საღებავებით და ...",NaN,NaN,35,გურიის


In [457]:
data.to_excel('test.xlsx', index=False)